In [1]:
import os
import math
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt
from weather.darksky import weather_request

In [2]:
def count_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: data[df_cols[i]].isnull().sum()})
    
    return dict_x

# Elia grid load data

In [15]:
## expanse by rows * columns
elia = pd.read_csv("ELIA_LOAD_2019.csv",header = 1)
elia.replace('NOT VALID', np.NaN, inplace=True)
elia = elia.iloc[:,:99]

date_list = []
for i in range(elia.shape[0]):
    date_list.append("{}-{}-{}".format(elia['yyyy'][i],elia['mm'][i],elia['dd'][i]))
elia["time"] = date_list

elia.drop(columns = ['dd','mm'], inplace = True)
elia.set_index(elia["time"], inplace= True)
elia.drop(columns = ['time'], inplace=True)

In [16]:
## fix 24:00 to the next day 00:00
elia['0:00'] = elia.iloc[:,-1].shift()
## swap
cols = list(elia.columns)
a, b = cols.index('yyyy'), cols.index('0:00')
cols[b], cols[a] = cols[a], cols[b]
elia = elia[cols]

elia.drop(columns=['24:00','yyyy'], inplace = True)


## Expanse the data set
index_list = []
energy_list = []
for date in elia.index:
    for time in elia.columns:
        #special case handling
        #if time == "24:00":
        #    index_list.append("{} {}".format(date , "00:00"))
        #else:
        index_list.append("{} {}".format(date, time))
        energy_list.append(elia.loc[date, time])
elia_qh = pd.DataFrame(list(zip(index_list,energy_list)),  columns=['time','grid_load'])
elia_qh.head()


##type transformation
elia_qh['time'] = elia_qh['time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M"))
elia_qh["grid_load"] = elia_qh["grid_load"].astype("float64")
elia_qh['time'] = elia_qh['time'].astype("datetime64")
elia_qh.dtypes
## index setting
elia_qh.set_index(elia_qh['time'], inplace = True)
elia_qh.drop(columns = ['time'], inplace = True)
elia_qh.to_csv("elia_qh.csv", index = True)


/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.


In [17]:
elia_qh = pd.read_csv("elia_qh.csv", index_col='time')
elia_qh.shape

(35040, 1)

### To prevent data leakage we split trian and test first

In [6]:
##8256
elia_train = elia_qh.iloc[:5781,:]
elia_test = elia_qh.iloc[5784:,:]
elia_train.shape, elia_test.shape

((5781, 1), (2472, 1))

**then deal with missing value**

In [25]:
## missing value train
#missing = count_missing(elia_train)
#print(missing)
## missing value test
#missing = count_missing(elia_test)
#print(missing)

missing = count_missing(elia_qh)
print(missing)

{'grid_load': 0}


In [18]:
#elia_train.fillna(elia_train.median(), inplace=True)
#elia_test.fillna(elia_test.median(), inplace=True)
elia_qh.fillna(elia_qh.median(), inplace = True)

**scale down the grid load**

In [9]:
elia_train['grid_load(MW)'] = elia_train['grid_load'].apply(lambda x: x/1000000)
elia_test['grid_load(MW)'] = elia_test['grid_load'].apply(lambda x: x/1000000)
elia_train.drop(['grid_load'],axis = 1, inplace = True)
elia_test.drop(['grid_load'],axis = 1, inplace = True)

/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//miniconda3/envs/tpot/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [19]:
elia_qh['grid_load(MW)'] = elia_qh['grid_load'].apply(lambda x: x/1000000)
elia_qh.drop(['grid_load'],axis = 1, inplace = True)

In [20]:
#elia_train.to_csv("elia_train.csv", index = True)
#elia_test.to_csv("elia_test.csv", index = True)
elia_qh.to_csv("elia_2019.csv", index = True)

### weather data from darksky

In [5]:
## request weather data in brussels
                    # lat   ,  long ,  start_date   , ndays, timezone      
wr = weather_request(50.8436, 4.3530, "2019-01-01 00:00", 370, 'CET')
wr.request()
weather_elia = wr.get_pandas()

In [11]:
weather_elia

,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone,precipAccumulation
time,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0376,0.02,7.49,5.01,4.43,0.81,1034.0,3.82,8.22,258,0.99,0,16.093,268.9,NaN
2019-01-01 01:00:00,0.0310,0.01,7.78,5.60,4.85,0.82,1033.9,3.39,8.07,250,0.99,0,16.093,265.3,NaN
2019-01-01 02:00:00,0.0242,0.01,7.38,5.00,4.56,0.82,1033.5,3.59,8.20,255,0.99,0,16.093,265.3,NaN
2019-01-01 03:00:00,0.0197,0.01,6.85,4.17,4.09,0.83,1032.9,3.92,8.31,257,0.99,0,16.093,266.0,NaN
2019-01-01 04:00:00,0.0198,0.01,6.37,3.45,3.64,0.83,1032.4,4.18,8.44,257,0.99,0,16.093,266.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-05 19:00:00,0.0125,0.01,5.96,3.53,1.31,0.72,1034.0,3.19,6.02,209,0.97,0,16.093,306.6,NaN
2020-01-05 20:00:00,0.0000,0.00,5.85,3.31,1.41,0.73,1033.8,3.31,5.80,205,0.97,0,16.093,300.6,NaN
2020-01-05 21:00:00,0.0000,0.00,5.81,3.13,1.66,0.75,1033.2,3.52,6.56,207,0.97,0,16.093,301.0,NaN


**Drop unused columns**

In [12]:
weather_elia.drop(columns = ['summary','icon','precipType','precipAccumulation'], axis = 1,inplace = True)

In [13]:
elia_w = weather_elia

In [15]:
elia_w_train = weather_elia.iloc[:1446,:]
elia_w_test = weather_elia.iloc[1446:]
elia_w_test.head()

,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone
time,,,,,,,,,,,,,,
2020-03-01 06:00:00,0.0531,0.23,5.11,-0.28,2.94,0.86,994.3,10.04,16.70,221,0.85,0,16.093,410.4
2020-03-01 07:00:00,0.0717,0.28,5.06,-0.28,3.02,0.87,994.4,9.80,15.50,223,0.88,0,16.093,411.1
2020-03-01 08:00:00,0.0895,0.31,5.18,0.04,3.06,0.86,994.5,9.22,14.30,222,0.92,0,16.093,409.6
2020-03-01 09:00:00,0.1070,0.27,5.41,0.30,3.26,0.86,994.7,9.41,14.79,223,0.95,0,16.093,409.2
2020-03-01 10:00:00,0.1214,0.23,5.84,0.82,3.43,0.85,994.8,9.55,14.96,224,0.95,1,16.093,408.0


**then deal with missing value**

In [14]:
missing = count_missing(elia_w)
print(missing)

{'precipIntensity': 0, 'precipProbability': 0, 'temperature': 0, 'apparentTemperature': 0, 'dewPoint': 0, 'humidity': 0, 'pressure': 0, 'windSpeed': 0, 'windGust': 0, 'windBearing': 0, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 0, 'ozone': 0}


In [16]:
## missing value train
missing = count_missing(elia_w_train)
print(missing)
## missing value test
missing = count_missing(elia_w_test)
print(missing)

{'precipIntensity': 0, 'precipProbability': 0, 'temperature': 0, 'apparentTemperature': 0, 'dewPoint': 0, 'humidity': 0, 'pressure': 0, 'windSpeed': 0, 'windGust': 0, 'windBearing': 0, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 0, 'ozone': 0}
{'precipIntensity': 0, 'precipProbability': 0, 'temperature': 0, 'apparentTemperature': 0, 'dewPoint': 0, 'humidity': 0, 'pressure': 0, 'windSpeed': 0, 'windGust': 0, 'windBearing': 0, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 0, 'ozone': 0}


### Merge weather and load

In [23]:
elia_2019 = elia_w.merge(elia_qh, how ='right', left_index = True, right_index = True)

In [17]:
train_elia =  elia_w_train.merge(elia_train, how ='right', left_index = True, right_index = True)
test_elia = elia_w_test.merge(elia_test, how = 'right', left_index = True, right_index = True)

In [24]:
elia_2019.head()

,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone,grid_load(MW)
time,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0376,0.02,7.49,5.01,4.43,0.81,1034.0,3.82,8.22,258.0,0.99,0.0,16.093,268.9,8.514733
2019-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.497139
2019-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.416679
2019-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.264418
2019-01-01 01:00:00,0.0310,0.01,7.78,5.60,4.85,0.82,1033.9,3.39,8.07,250.0,0.99,0.0,16.093,265.3,8.133287


**then deal with missing value by interpolation**

In [25]:
for col in elia_2019.columns:
    elia_2019[col] = elia_2019[col].interpolate(method='linear')

In [19]:
## interpolate nan
for col in train_elia.columns:
    train_elia[col] = train_elia[col].interpolate(method='linear')
for col in test_elia.columns:
    test_elia[col] = test_elia[col].interpolate(method = "linear")

In [26]:
elia_2019.to_csv("elia_2019.csv",index = True)

In [20]:
train_elia.to_csv("train_elia.csv",index = True)
test_elia.to_csv("test_elia.csv", index = True)

# London electricity consumption smart meter

In [16]:
elec = pd.read_csv("../smart-meters-in-london/halfhourly_dataset/block_0.csv")
for num in range(1,111):    
    elec = elec.append(pd.read_csv("../smart-meters-in-london/halfhourly_dataset/block_"+str(num)+".csv"))
        

/Users/fuchucheng/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
elec.replace('Null', np.NaN, inplace=True)
energy = elec.iloc[:,:]
energy["energy"] = energy["energy(kWh/hh)"].astype("float64")
energy = energy.drop(["energy(kWh/hh)"],axis = 1)

**Missing values**

In [18]:
missing = missing_count(energy)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 5544), ('LCLid', 0), ('tstp', 0)]

### Single household

In [20]:
energy_MAC1 = energy[energy["LCLid"] == "MAC000010"]

In [21]:
missing = missing_count(energy_MAC1)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]

Count of missing data


[('energy', 1), ('LCLid', 0), ('tstp', 0)]

In [22]:
energy_MAC1["energy"] = energy_MAC1["energy"].astype("float64")
energy_MAC1["tstp"] = energy_MAC1["tstp"].astype("datetime64")
energy_MAC1 = energy_MAC1.groupby('tstp').agg({'energy': lambda x: x.mean(skipna=False)})
energy_MAC1['tstp'] = energy_MAC1.index
energy_MAC1.drop(['tstp'], axis = 1, inplace=True)


/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/fuchucheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### To prevent data leakage we split trian and test first

In [30]:
train_s = energy_MAC1.iloc[:20000, :]
test_s = energy_MAC1.iloc[20000:,:]

**then deal with missing value half-hr**

In [31]:
train_s.fillna(train_s.median(), inplace = True)
test_s.fillna(test_s.median(), inplace = True)

**resampling to 1hr**

In [32]:
train_s_h = train_s.resample('1H').sum()
test_s_h = test_s.resample('1H').sum()

In [36]:
##Save to csv
train_s.to_csv("train_s_hh.csv", index = True)
test_s.to_csv("train_s_hh.csv", index = True)
train_s_h.to_csv("train_s_h.csv", index = True)
test_s_h.to_csv("train_s_h.csv", index = True)

### Aggregated   household

**Group by LCLid and aggregate over per hour**


In [29]:
## deal with missing value by interpolating mean of whole column
# energy["energy"].fillna(energy["energy"].median(), inplace = True)
## group by time stamps and aggregate through mean
energy["energy"] = energy["energy"].astype("float64")
energy["tstp"] = energy["tstp"].astype("datetime64")
energy = energy.groupby('tstp').agg({'energy': lambda x: x.mean(skipna=False)})
energy['tstp'] = energy.index
energy.drop(['tstp'], axis = 1, inplace=True)

### To prevent data leakage we split trian and test first

In [6]:
train_e = energy.iloc[:28750, :]
test_e = energy.iloc[28750:,:]


**deal with missing value sampling rate: 30 mins**

In [7]:
train_e.fillna(train_e.median(), inplace = True)
test_e.fillna(test_e.median(), inplace = True)


//miniconda3/envs/tpot/lib/python3.6/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
## check if there is still missing value
missing = missing_count(test_e)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 0)]

**then deal with missing value and resampling to 1hr**

In [31]:
train_e.fillna(train_e.median(), inplace = True)
test_e.fillna(test_e.median(), inplace = True)
train_e = train_e.resample('1H').sum()
test_e = test_e.resample('1H').sum()

//miniconda3/envs/TPOT/lib/python3.5/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
## check if there is still missing value
missing = missing_count(train_e)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 673)]

### Weather data processing


In [9]:
weather = pd.read_csv('../smart-meters-in-london/weather_hourly_darksky.csv')

In [10]:
## set index as time stamp
weather = weather.set_index("time")
weather.index = weather.index.astype("datetime64[ns]")

In [11]:
## apparently the sequence in between is not correct
weather = weather.sort_index()

**Missing values**

In [12]:
missing = count_missing(weather)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:]

Percent of missing data


[('pressure', 1300),
 ('visibility', 0),
 ('windBearing', 0),
 ('temperature', 0),
 ('dewPoint', 0),
 ('apparentTemperature', 0),
 ('windSpeed', 0),
 ('precipType', 0),
 ('icon', 0),
 ('humidity', 0),
 ('summary', 0)]

In [13]:
## drop columns wont be used later
weather.drop(['precipType','icon','summary'], axis = 1, inplace=True)

In [14]:
##merge trian and test seperately
train =  weather.merge(train_e, how ='right', left_index = True, right_index = True)
test = weather.merge(test_e.iloc[1:,:], how = 'right', left_index = True, right_index = True)

In [16]:
train.to_csv("hh_train_nan.csv", index = True)
test.to_csv("hh_test_nan.csv", index = True)

In [12]:
train = pd.read_csv("hh_train_nan.csv", index_col='tstp')
test = pd.read_csv("hh_test_nan.csv", index_col='tstp')

In [39]:
## interpolate nan
for col in train.columns:
    train[col] = train[col].interpolate(method='linear')
for col in test.columns:
    test[col] = test[col].interpolate(method = "linear")

In [43]:
## data set without catagorical features
train.to_csv("train_nocat_hh.csv", index = True)
test.to_csv("test_nocat_hh.csv", index = True)

**One hot encoding**

In [73]:
## Examine unique categories in categorical features
p = list(weather["precipType"].unique()) 
i = list(weather["icon"].unique()) 
u = list(weather["summary"].unique())
# Sort them for one hot encoding labels
p.sort()
i.sort()
u.sort()

In [74]:
from sklearn.preprocessing import OneHotEncoder
## One hot encoding these features above
onehot = OneHotEncoder(dtype=np.int, sparse=True)
nominals = pd.DataFrame(onehot.fit_transform(weather[['precipType', 'icon','summary']]).toarray(),columns=p+i+u)
nominals.index = weather.index

In [12]:
#Concate nominals with origin weather dataframe
weather = weather.set_index("time")
weather.index = weather.index.astype("datetime64")
#weather.drop(["precipType","icon","summary"],axis = 1, inplace = True)
weather =  weather.merge(nominals, left_index = True, right_index = True)


KeyError: "None of ['time'] are in the columns"